# <h1 align="center"><font color="red">How to Build a Simple Reasoning and Acting Agent from Scratch</font></h1>

<font color="yellow">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este Notebook está baseado no Tutorial de [kirouane Ayoub]().

Links de estudo:

* [Part 1](https://blog.gopenai.com/introduction-to-llm-agents-how-to-build-a-simple-reasoning-and-acting-agent-from-scratch-part-1-843e14686be7)

* [Part 2](https://blog.gopenai.com/building-llm-agents-from-scratch-part-2-a-conversational-search-agent-with-ollama-a4544b2291cf)

# <font color="gree">Contextualizando</font>

Esta Notebook inicia uma série sobre a construção de `agentes de IA´, começando com uma implementação básica usando `Ollama´ para executar `LLMs´ localmente. O objetivo é criar um sistema que compreenda consultas do usuário e utilize pesquisa na web para fornecer respostas informativas, demonstrando os conceitos fundamentais do design de agentes de IA.

Esta implementação serve como um ponto de partida para entender o desenvolvimento de `agentes de IA`, mas é uma versão simplificada. Em cenários reais, os agentes teriam comportamentos mais complexos, exigindo técnicas de análise sintática robustas para lidar com saídas inesperadas. Além disso, seriam necessárias estratégias abrangentes de tratamento de erros para garantir uma operação suave e confiável do sistema.

# <font color="gree">Funcionamento do Agent</font>

<img src="https://miro.medium.com/v2/resize:fit:828/format:webp/1*-c8XfC8zGlrmF8DMwWkJaA.png" alt="Minha Imagem" width="700" height="400"/>


* `Agent:` Representa o sistema geral que abrange o LLM e suas interações com o ambiente.

* `Task:` Este é o objetivo ou meta que o agente está tentando atingir. É o ponto de partida do processo.

* `LLM:` Este é o núcleo do agente, um LLM que processa informações e toma decisões.

* `Reasoning:` O LLM utiliza o raciocínio para determinar o melhor curso de ação com base na tarefa e nas informações disponíveis.

* `Tools:` O agente pode utilizar várias ferramentas (`como pesquisa na web`, `calculadoras` ou `bancos de dados`) para coletar informações adicionais ou executar ações.

* `Action:` Com base em seu raciocínio, o LLM decide uma ação a ser tomada em seu ambiente.

* `Environment:` Representa o mundo externo ou contexto no qual o agente opera. `Pode ser um site`, `um espaço físico` ou `um ambiente simulado`.

* `Result:` A ação tomada pelo agente produz um resultado no ambiente, que então realimenta a compreensão e as ações futuras do agente.


# <font color="gree">Exemplo de código</font>

## <font color="yellow">Configurando Ollama e Dependências</font>

```
curl -fsSL https://ollama.com/install.sh | sh

ollama serve
ollama pull llama3.1
```

Também:

```
pip install openai duckduckgo-search
```

## <font color="yellow">Criando um cliente OpenAI para interação LLM local</font>

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
Eddy_key_openai  = os.environ['OPENAI_API_KEY']

from openai import OpenAI
client = OpenAI(api_key=Eddy_key_openai,
                base_url="http://127.0.0.1:11434/v1"
                )

## <font color="yellow">Implementando a funcionalidade de pesquisa</font>

Nosso agente precisa da capacidade de pesquisar informações na `web`. Definimos uma função `search` que utiliza a biblioteca `duckduckgo-search` para consultar `DuckDuckGo` e recuperar resultados de pesquisa relevantes.



`Esses resultados serão usados ​​para aumentar o conhecimento do LLM quando necessário.`

In [2]:
from duckduckgo_search import DDGS

def search(query , max_results=3):
  results = DDGS().text(query, max_results=max_results)

  return str("\n".join(str(results[i]) for i in range(len(results))))


## <font color="yellow">Definindo ação e extração de entrada</font>

Para orientar o comportamento do agente, precisamos entender suas ações pretendidas com base na saída do `LLM`.


Definimos uma função para extrair a `“Action”` e a `“Action Input”` da resposta do `LLM`, permitindo-nos determinar se ele pretende responder diretamente ao usuário ou realizar uma pesquisa na web.

In [3]:
import re
def extract_action_and_input(text):
  action_pattern = r"Action: (.+?)\n"
  input_pattern = r"Action Input: \"(.+?)\""
  action = re.findall(action_pattern, text)
  action_input = re.findall(input_pattern, text)
  return action, action_input


## <font color="yellow">Elaborando o Prompt do Sistema</font>

O prompt do sistema (`system prompt`) fornece instruções ao `LLM` sobre como se comportar. Ele descreve as ferramentas disponíveis (`neste caso, pesquisa na web`) e o processo de tomada de decisão para escolher entre respostas diretas e utilizar a ferramenta de pesquisa.


`Este prompt garante que o agente siga um fluxo de trabalho lógico.`

In [ ]:
System_prompt = """
Responda às seguintes perguntas da melhor forma possível. Use a pesquisa na web somente se necessário. 

Você tem acesso à seguinte ferramenta: 

Pesquisa: útil para quando você precisa responder a perguntas sobre eventos atuais ou quando precisa de informações específicas que não conhece. 
          No entanto, você só deve usá-la se a consulta do usuário não puder ser respondida com conhecimento geral ou uma resposta direta. 

Você receberá uma mensagem do humano, então você deve iniciar um loop e fazer um dos seguintes: 

Opção 1: Responder ao humano diretamente 
         - Se a pergunta do usuário for simples, coloquial ou puder ser respondida com seu conhecimento existente, responda diretamente. 
         - Se o usuário não solicitou uma pesquisa ou a pergunta não precisa de informações atualizadas ou específicas, evite usar a ferramenta de pesquisa. 

Use o seguinte formato ao responder ao humano: 
Pensamento: Explique por que uma resposta direta é suficiente ou por que nenhuma pesquisa é necessária. 
Ação: Resposta ao Humano
Entrada de ação: "sua resposta ao humano, resumindo o que você sabe ou concluindo a conversa" 

Opção 2: Use a ferramenta de pesquisa para responder à pergunta. 
         - Se precisar de mais informações para responder à pergunta, ou se a pergunta for sobre um evento atual ou conhecimento específico que pode não estar nos seus dados de treinamento, use a ferramenta de pesquisa. 
         - Após receber os resultados da pesquisa, decida se você tem informações suficientes para responder à pergunta ou se outra pesquisa é necessária. 

Use o seguinte formato ao usar a ferramenta de pesquisa: 
Pensamento: Explique por que uma pesquisa é necessária ou por que pesquisas adicionais são necessárias. 
Ação: Pesquisar 
ação Entrada: "a entrada para a ação, a ser enviada para a ferramenta" 

Lembre-se de sempre decidir cuidadosamente se deve pesquisar ou responder diretamente. Se puder responder à pergunta sem pesquisar, sempre prefira responder diretamente. 
"""
